In [6]:
#Импорты
import pandas as pd
import re 

In [7]:
#Читаем датасет
rating_movies = pd.read_csv('data/movies_data/ratings_movies.csv')
display(rating_movies)

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [8]:

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [18]:
#1
rating_movies['year_release'] = rating_movies['title'].apply(get_year_release)
rating_movies['year_release'] = rating_movies['year_release'].astype('Int64')
rating_movies['date'] = pd.to_datetime(rating_movies['date'])
display(rating_movies.info())
display(rating_movies)
#2
display(rating_movies[rating_movies['year_release'] == 1999].sort_values(by=['rating', 'title']).head(1))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  Int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 6.3+ MB


None

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995
...,...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017


,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
47107,47107,307,2572,0.5,2007-08-02 19:50:48,10 Things I Hate About You (1999),Comedy|Romance,1999
